In [2]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.8 MB/s eta 0:00:00


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# autoconfig and autotokenizer are classes provided by Hugging Face 'transformers'
# library, powerful library for NLP
!pip install sentencepiece
!pip install sacremoses
!pip install importlib_metadata
!pip install transformers

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer
from transformers import AutoConfig

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.3 MB/s eta 0:00:00


In [8]:
# this the model we want to use:
# model created by Google for NLP, base model so easy to deploy
# sometimes not best performance, uncased means casing of words
# doesn't matter

model_name = 'prajjwal1/bert-small'
num_labels = 5

# load the pre-trained model
model = torch.hub.load('huggingface/pytorch-transformers', 'model', model_name)

# download model and configuration from s3 and cache

# autoconfig - This class is used to load the configuration for a specific pre-trained model.
# The AutoConfig class provides a way to automatically load the appropriate configuration for a 
# given pre-trained model, so you don't have to manually specify all the details (layers, hidden sizes, etc.).
# allows you to specify parts you need to.

config = AutoConfig.from_pretrained(model_name)

# getting hidden-size value from the config object and assigns to last_state_dim
# hidden_size normally represents dimensionality of hidden states in nn
last_state_dim = config.hidden_size

# replace the pre trained head w a new one
# allows model to adapt to the specific classification task w num_labels output classes
# model.classifier refers to the head of the model responsible for making predictions.
# it's replaced with new linear layer that tasks last_state_dim as the input dimension
# and num_labels as output dimension
model.classifier = torch.nn.Linear(last_state_dim, num_labels)

# Define your loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# load tokenizer
# converting raw text into numerical tokens that the model can understand
tokenizer = AutoTokenizer.from_pretrained(model_name)

# preprocess the input text
# padding adds padding tokens to make all input sequences the same length
# truncation truncates input sequences to a max length by tokenizr
# return_tensors returns PyTorch tensors instead of regular Python lists
def preprocess(text):
    # tokenization result is stored in the inputs variable, which is a dictionary-like
    # object containing tokenized inputs. The 'input_ids' key corresponds to the tokenized
    # input sequence and 'attention_mask' represents a binary mask that indicated which 
    # tokens are actual words and which ones are actual tokens
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    return inputs['input_ids'], inputs['attention_mask']

# perform sentiment analysis
def predict_sentiment(texts):
    input_ids, attention_mask = preprocess(texts)

    with torch.no_grad():
        # invokes forward pass, which processes the input sequence and generates
        # hidden states
        hidden_states = model(input_ids, attention_mask=attention_mask)
        # hidden_states is an object that provides access to the last hidden state
        # of the model. the last one contains rich info about the input sequence
        last_hidden_states = hidden_states.last_hidden_state
        final_feature_map = last_hidden_states[:,-1,:]

    # pass the hidden states through the classifier
    logits = model.classifier(final_feature_map)
    # turning real numbers into probability distrbution between 1 and 0
    # represents the probability of the corresponding class being the correct prediction
    probabilities= F.softmax(logits, dim=1)
    # stores predicted class labels based on computed probabilities
    sentiments = torch.argmax(probabilities, dim=1).squeeze()
    
    return sentiments, probabilities

# load in datasets
train_data = dataset['train']
train_texts = train_data['text']
train_labels = train_data['label']

# Number of training iterations
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    for text, label in zip(train_texts, train_labels):
        model.train()  # Set the model to training mode

        # Preprocess the input text
        input_ids, attention_mask = preprocess(text)

        optimizer.zero_grad()  # Clear gradients

        # Forward pass
        hidden_states = model(input_ids, attention_mask=attention_mask)
        last_hidden_states = hidden_states.last_hidden_state
        final_feature_map = last_hidden_states[:, -1, :]
        logits = model.classifier(final_feature_map)

        # Compute loss
        loss = criterion(logits, torch.tensor([label]))

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_texts)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {average_loss}")

# cvall the predict_sentiment function
text = "This is a great movie!"
sentiment, probabilities = predict_sentiment(text)

print("Sentiment: {sentiment}")
print("Probabilities: {probabilities}")
        
# Load the test data
test_data = dataset['test']
test_texts = test_data['text']
test_labels = test_data['label']

# Evaluation loop
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0

for text, label in zip(test_texts, test_labels):
    input_ids, attention_mask = preprocess(text)

    with torch.no_grad():
        hidden_states = model(input_ids, attention_mask=attention_mask)
        last_hidden_states = hidden_states.last_hidden_state
        final_feature_map = last_hidden_states[:, -1, :]
        logits = model.classifier(final_feature_map)
        probabilities = F.softmax(logits, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1).squeeze()

    total_samples += 1
    if predicted_labels == label:
        total_correct += 1

accuracy = total_correct / total_samples
print("Test Accuracy:", accuracy)


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but n

RuntimeError: ignored